In [119]:
import os
import csv
import collections
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist 
import sys

file_dir_train = os.path.join(os.getcwd(), 'train.csv')
df_train = pd.read_csv(file_dir_train)
df_train=df_train.iloc[::2, :]
X_train = np.array(df_train.iloc[:,1:])


In [70]:
from scipy.spatial.distance import cdist 

def kmeans(data, k=3):
    centroid_indexes = np.random.choice(len(data), k, replace=False)
    
    centroids = data[centroid_indexes, :]

    distances= cdist(data, centroids ,'euclidean')
    points = np.array([np.argmin(i) for i in distances])
    
    centroids_final = []
    new_points_with_labels = np.ones_like(points)
    number_of_differences_in_labels = np.sum(points != new_points_with_labels)
    number_of_iteration=0
    i =0
    while(number_of_differences_in_labels > 2 and number_of_iteration <1000):
        number_of_iteration= number_of_iteration + 1
        centroids = []
        new_points_with_labels = points.copy()
        for centroid_indexes in range(k):
            temp_cent = data[points==centroid_indexes].mean(axis=0) 
            centroids.append(temp_cent)
 
        centroids = np.vstack(centroids)
        distances = cdist(data, centroids ,'euclidean')
        points = np.array([np.argmin(i) for i in distances])
        number_of_differences_in_labels = np.sum(points != new_points_with_labels)
        centroids_final = centroids.copy()
        
    return centroids_final, points 

In [144]:
def Davies_Bouldin(data, clusters):
    count_cluster = len(np.bincount(clusters))
    cluster_i = [data[clusters == i] for i in range(count_cluster)]
    centroid = [np.mean(i, axis = 0) for i in cluster_i]
    variance = [np.mean([euclidean_distance(p, centroid[i]) for p in j]) for i, j in enumerate(cluster_i)]
    DB = []

    for i in range(count_cluster):
        for j in range(count_cluster):
            if j != i:
                DB.append((variance[i] + variance[j]) / euclidean_distance(centroid[i], centroid[j]))

    return (np.max(DB) / count_cluster)

def euclidean_distance(data1, data2):
    return np.sqrt(np.sum(np.square(data1 - data2)))

def C_index(data, clusters, distance_between_each_point):
    distances_between_all_pairs_in_each_pattern= []
    number_of_pairs_in_each_pattern = []
    count_cluster = len(np.bincount(clusters))
 
    cluster_i = [data[clusters == i] for i in range(count_cluster)]
    
    
    for i in cluster_i:
        n_elements = len(i)
        for k in range (n_elements):
            for j in range ( k+1,n_elements):
                distances_between_all_pairs_in_each_pattern.append(np.linalg.norm(X_train[k] - X_train[j]))
        number_of_pairs_in_each_pattern.append(((n_elements - 1)* n_elements) /2)
    
    G = sum(distances_between_all_pairs_in_each_pattern)
    
    alpha = sum(number_of_pairs_in_each_pattern)
    alpha = int(alpha)
    min_sum_val = sum(distance_between_each_point[0:int(max(number_of_pairs_in_each_pattern))])
    max_sum_val = sum(distance_between_each_point[-int(max(number_of_pairs_in_each_pattern)):])
    return ((G-min_sum_val)/(max_sum_val-min_sum_val))

    


In [145]:
distance_between_each_point = []
length =len(X_train)
for i in range (length):
    for j in range ( i+1,length):
        distance_between_each_point.append(np.linalg.norm(X_train[i] - X_train[j]))
        
distance_between_each_point.sort()
print(distance_between_each_point)

[465.41486869243874, 502.91549986056305, 534.3173214485939, 535.370899470638, 556.4692264627039, 556.5662584095446, 580.4239829641776, 598.9866442584508, 602.1087941560063, 643.2021455188096, 647.2101667928279, 648.8397336785101, 657.0448995312269, 658.3304641287687, 679.2731409381648, 685.1394018738084, 685.6660994974158, 693.3895009300328, 710.4498574846785, 712.2815454579741, 731.9699447381703, 735.06598343278, 745.8035934480338, 753.3644536344942, 754.7489648883263, 761.4374563941545, 768.3280549348696, 768.4412794742354, 770.5874382573337, 774.889024312514, 777.56028705175, 781.6591328705883, 786.069335364254, 787.1334067361136, 791.4000252716701, 794.3519371160368, 795.6368518363136, 797.1154245151702, 797.1787754324622, 797.9442336404218, 816.126828379021, 816.6823127752921, 817.2080518448163, 817.9523213488669, 818.2407958541299, 818.75209923395, 823.434271815304, 824.077666242691, 825.1957343564011, 831.2941717587103, 832.2836055095643, 832.8697377141277, 833.9352492849789, 83

In [147]:
if __name__ == "__main__":
    K = [5, 7, 9, 10, 12,15]
    
    accuracy  = []

    for k in K:
        labels = kmeans(X_train, k)
        print("k = " + str(k))
        print("David Bouldin Score: " + str(Davies_Bouldin(X_train, labels[1])))
        print("C index: " + str(C_index(X_train, labels[1], distance_between_each_point)))

k = 5
David Bouldin Score: 0.59829698382006
C index: 0.22980712432378728
k = 7
David Bouldin Score: 0.39607601368787854
C index: 0.19239205459271544
k = 9
David Bouldin Score: 0.3394030226231949
C index: 0.20351281129133644
k = 10
David Bouldin Score: 0.2814702256067801
C index: 0.22248910006260073
k = 12
David Bouldin Score: 0.2712729214495017
C index: 0.1720930176035548
k = 15
David Bouldin Score: 0.20710696743213383
C index: 0.14528711255376903
